In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import wandb

# Tensorflow

### WandB connection

In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: filippetran (ns-super-team). Use `wandb login --relogin` to force relogin


True

### Loading the data

In [3]:
x_train = pd.read_csv('data/train/data.csv').to_numpy(dtype='float32')
y_train = pd.read_csv('data/train/labels.csv').to_numpy(dtype='float32')

x_val = pd.read_csv('data/val/data.csv').to_numpy(dtype='float32')
y_val = pd.read_csv('data/val/labels.csv').to_numpy(dtype='float32')

print(x_train[0])
print(y_train[0])
print()
print(x_val[0])
print(y_val[0])

[  1.      49.       1.       1.     110.8833]
[0.]

[ 1. 39.  0.  0. 50.]
[0.]


### Building a model

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(5,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(48, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')    
    ])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                192       
                                                                 
 dense_1 (Dense)             (None, 48)                1584      
                                                                 
 dense_2 (Dense)             (None, 32)                1568      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3,377
Trainable params: 3,377
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

### Training the model

In [7]:
model.fit(
    x_train,
    y_train,
    epochs=500,
    validation_data=(x_val, y_val)
    )

Epoch 1/500
20/20 [==============================] - 2s 39ms/step - loss: 0.9428 - accuracy: 0.5823 - val_loss: 0.7339 - val_accuracy: 0.6391
Epoch 2/500
20/20 [==============================] - 0s 10ms/step - loss: 0.6611 - accuracy: 0.6484 - val_loss: 0.6030 - val_accuracy: 0.6992
Epoch 3/500
20/20 [==============================] - 0s 9ms/step - loss: 0.6153 - accuracy: 0.6952 - val_loss: 0.6287 - val_accuracy: 0.6466
Epoch 4/500
20/20 [==============================] - 0s 8ms/step - loss: 0.6141 - accuracy: 0.6742 - val_loss: 0.6205 - val_accuracy: 0.6917
Epoch 5/500
20/20 [==============================] - 0s 7ms/step - loss: 0.5987 - accuracy: 0.6903 - val_loss: 0.5967 - val_accuracy: 0.6917
Epoch 6/500
20/20 [==============================] - 0s 10ms/step - loss: 0.6008 - accuracy: 0.6952 - val_loss: 0.5919 - val_accuracy: 0.6617
Epoch 7/500
20/20 [==============================] - 0s 8ms/step - loss: 0.6012 - accuracy: 0.7097 - val_loss: 0.6076 - val_accuracy: 0.6917
Epoch 8/50

### Validation